In [22]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from datetime import datetime

# 创建输出目录，如果不存在
output_dir = 'output_directory'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 生成带时间戳的文件名
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
output_file = os.path.join(output_dir, f'DiD_model_results_{timestamp}.xlsx')

# 读取交通事故数据
traffic_data_file = 'TrafficAccident.xlsx'  # 根据实际路径修改
traffic_df = pd.read_excel(traffic_data_file)

# 检查交通事故数据的列名和空值
print("Traffic data columns:", traffic_df.columns)
print("Missing values in traffic data:\n", traffic_df.isnull().sum())


Traffic data columns: Index(['COD_DIS', 'COD_BAR', 'NOMBRE_BAR', '201801_rate', '201802_rate',
       '201803_rate', '201804_rate', '201805_rate', '201806_rate',
       '201807_rate', '201808_rate', '201809_rate', '201810_rate',
       '201811_rate', '201812_rate', '201901_rate', '201902_rate',
       '201903_rate', '201904_rate', '201905_rate', '201906_rate',
       '201907_rate', '201908_rate', '201909_rate', '201910_rate',
       '201911_rate', '201912_rate', '202001_rate', '202002_rate',
       '202003_rate', '202004_rate', '202005_rate', '202006_rate',
       '202007_rate', '202008_rate', '202009_rate', '202010_rate',
       '202011_rate', '202012_rate', '202101_rate', '202102_rate',
       '202103_rate', '202104_rate', '202105_rate', '202106_rate',
       '202107_rate', '202108_rate', '202109_rate', '202110_rate',
       '202111_rate', '202112_rate', '202201_rate', '202202_rate',
       '202203_rate', '202204_rate', '202205_rate', '202206_rate',
       '202207_rate', '202208_rate

In [23]:
control_data_file = 'CONTROL_ADD_PAND.xlsx'  # 根据实际路径修改
education_df = pd.read_excel(control_data_file, sheet_name='EDUCATION')
income_df = pd.read_excel(control_data_file, sheet_name='INCOME')
population_df = pd.read_excel(control_data_file, sheet_name='POPULATION')
transport_df = pd.read_excel(control_data_file, sheet_name='TRANSPORT')
commercial_df = pd.read_excel(control_data_file, sheet_name='COMMERCIAL')
residential_df = pd.read_excel(control_data_file, sheet_name='RESIDENTIAL')
PAND_df = pd.read_excel(control_data_file, sheet_name='PANDEMIC')


# 检查每个社会因子数据的列名和空值情况
dataframes = {
    "EDUCATION": education_df,
    "INCOME": income_df,
    "POPULATION": population_df,
    "TRANSPORT": transport_df,
    "COMMERCIAL": commercial_df,
    "RESIDENTIAL": residential_df,
    "PANDEMIC": PAND_df
}

# 打印每个工作表的列名和空值数量
for sheet_name, df in dataframes.items():
    print(f"\n{sheet_name} data columns:", df.columns)
    print(f"Missing values in {sheet_name} data:\n", df.isnull().sum())




EDUCATION data columns: Index(['COD_DIS', 'COD_BAR', 'NOMBRE_BAR', 'EDUCATION_2018_01',
       'EDUCATION_2018_02', 'EDUCATION_2018_03', 'EDUCATION_2018_04',
       'EDUCATION_2018_05', 'EDUCATION_2018_06', 'EDUCATION_2018_07',
       'EDUCATION_2018_08', 'EDUCATION_2018_09', 'EDUCATION_2018_10',
       'EDUCATION_2018_11', 'EDUCATION_2018_12', 'EDUCATION_2019_01',
       'EDUCATION_2019_02', 'EDUCATION_2019_03', 'EDUCATION_2019_04',
       'EDUCATION_2019_05', 'EDUCATION_2019_06', 'EDUCATION_2019_07',
       'EDUCATION_2019_08', 'EDUCATION_2019_09', 'EDUCATION_2019_10',
       'EDUCATION_2019_11', 'EDUCATION_2019_12', 'EDUCATION_2020_01',
       'EDUCATION_2020_02', 'EDUCATION_2020_03', 'EDUCATION_2020_04',
       'EDUCATION_2020_05', 'EDUCATION_2020_06', 'EDUCATION_2020_07',
       'EDUCATION_2020_08', 'EDUCATION_2020_09', 'EDUCATION_2020_10',
       'EDUCATION_2020_11', 'EDUCATION_2020_12', 'EDUCATION_2021_01',
       'EDUCATION_2021_02', 'EDUCATION_2021_03', 'EDUCATION_2021_04',
 

In [24]:
# 读取政策处理信息
did_data_file = 'did_input_data_monthly.xlsx'  # 根据实际路径修改
did_df = pd.read_excel(did_data_file)

# 检查政策数据的列名和空值
print("Did data columns:", did_df.columns)
print("Missing values in did data:\n", did_df.isnull().sum())

Did data columns: Index(['COD_DIS', 'COD_BAR', 'NOMBRE_BAR', 'year', 'month', 'treati', 'postit',
       'treat*postit', 'DID'],
      dtype='object')
Missing values in did data:
 COD_DIS         0
COD_BAR         0
NOMBRE_BAR      0
year            0
month           0
treati          0
postit          0
treat*postit    0
DID             0
dtype: int64


In [25]:
# 打印列名，检查数据列
print("Columns in traffic_df before melt:", traffic_df.columns)

# 确保在melt时将所有以 '201801_rate' 格式命名的列都转换为 Date 和 accident_rate
traffic_df = traffic_df.melt(id_vars=['COD_DIS', 'COD_BAR'], 
                             value_vars=[col for col in traffic_df.columns if '_rate' in col],  # 选择包含 '_rate' 的列
                             var_name='Date', value_name='accident_rate_melted')

# 检查melt后的数据
print("Traffic Data after melt:")
print(traffic_df.head())

# 将 'YYYYMM_rate' 转化为 'YYYY-MM' 格式的日期
traffic_df['Date'] = traffic_df['Date'].str[:4] + '-' + traffic_df['Date'].str[4:6]

# 检查转换后的日期格式
print("Traffic Data after date conversion:")
print(traffic_df.head())


Columns in traffic_df before melt: Index(['COD_DIS', 'COD_BAR', 'NOMBRE_BAR', '201801_rate', '201802_rate',
       '201803_rate', '201804_rate', '201805_rate', '201806_rate',
       '201807_rate', '201808_rate', '201809_rate', '201810_rate',
       '201811_rate', '201812_rate', '201901_rate', '201902_rate',
       '201903_rate', '201904_rate', '201905_rate', '201906_rate',
       '201907_rate', '201908_rate', '201909_rate', '201910_rate',
       '201911_rate', '201912_rate', '202001_rate', '202002_rate',
       '202003_rate', '202004_rate', '202005_rate', '202006_rate',
       '202007_rate', '202008_rate', '202009_rate', '202010_rate',
       '202011_rate', '202012_rate', '202101_rate', '202102_rate',
       '202103_rate', '202104_rate', '202105_rate', '202106_rate',
       '202107_rate', '202108_rate', '202109_rate', '202110_rate',
       '202111_rate', '202112_rate', '202201_rate', '202202_rate',
       '202203_rate', '202204_rate', '202205_rate', '202206_rate',
       '202207_rate',

In [26]:
def merge_social_factors(df, social_factor_df, factor_name):
    # 检查传入的 df 是否为 None
    if df is None:
        raise ValueError("The main DataFrame 'df' cannot be None")
    
    # 找出所有包含日期信息的列（假设列名格式为 '因子名称_YYYY_MM'）
    date_columns = [col for col in social_factor_df.columns if '_' in col and len(col.split('_')) == 3]  # 查找列名中包含'_'并且长度为3的列
    
    # 打印调试信息，查看哪些列符合预期格式
    print(f"Date columns in {factor_name} data:", date_columns)
    
    # 创建一个空的 DataFrame 用于存储处理后的数据
    melted_df = pd.DataFrame()
    
    # 遍历所有列
    for col in date_columns:
        # 提取列名中的因子名称和日期部分
        factor, date_str = col.split('_')[0], col.split('_')[1] + '-' + col.split('_')[2]
        
        # 生成一个新的 DataFrame，包含 'COD_DIS', 'COD_BAR', 'Date', 和因子值
        temp_df = social_factor_df[['COD_DIS', 'COD_BAR', col]].copy()
        temp_df['Date'] = date_str  # 添加日期列
        temp_df.rename(columns={col: factor_name}, inplace=True)  # 重命名因子值列
        
        # 将这个 DataFrame 合并到结果中
        melted_df = pd.concat([melted_df, temp_df], ignore_index=True)

    # 检查社会因子数据是否有缺失值
    print(f"Missing values in {factor_name} data:\n", melted_df.isnull().sum())
    
    # 确保合并时列一致
    print(f"Columns in df before merge: {df.columns}")
    print(f"Columns in melted_df before merge: {melted_df.columns}")
    
    # 合并到主数据集
    df = pd.merge(df, melted_df, on=['COD_DIS', 'COD_BAR',  'Date'], how='left')
    
    print(f"Columns in df after merge: {df.columns}")  # 打印合并后的列名
    
    return df

# 确保 traffic_df 是有效的 DataFrame
if traffic_df is None:
    print("Error: traffic_df is None")
else:
    traffic_df = merge_social_factors(traffic_df, education_df, 'EDUCATION')
    traffic_df = merge_social_factors(traffic_df, income_df, 'INCOME')
    traffic_df = merge_social_factors(traffic_df, population_df, 'POPULATION')
    traffic_df = merge_social_factors(traffic_df, transport_df, 'TRANSPORT')
    traffic_df = merge_social_factors(traffic_df, commercial_df, 'COMMERCIAL')
    traffic_df = merge_social_factors(traffic_df, residential_df, 'RESIDENTIAL')
    traffic_df = merge_social_factors(traffic_df, PAND_df, 'PANDEMIC')

        

Date columns in EDUCATION data: ['EDUCATION_2018_01', 'EDUCATION_2018_02', 'EDUCATION_2018_03', 'EDUCATION_2018_04', 'EDUCATION_2018_05', 'EDUCATION_2018_06', 'EDUCATION_2018_07', 'EDUCATION_2018_08', 'EDUCATION_2018_09', 'EDUCATION_2018_10', 'EDUCATION_2018_11', 'EDUCATION_2018_12', 'EDUCATION_2019_01', 'EDUCATION_2019_02', 'EDUCATION_2019_03', 'EDUCATION_2019_04', 'EDUCATION_2019_05', 'EDUCATION_2019_06', 'EDUCATION_2019_07', 'EDUCATION_2019_08', 'EDUCATION_2019_09', 'EDUCATION_2019_10', 'EDUCATION_2019_11', 'EDUCATION_2019_12', 'EDUCATION_2020_01', 'EDUCATION_2020_02', 'EDUCATION_2020_03', 'EDUCATION_2020_04', 'EDUCATION_2020_05', 'EDUCATION_2020_06', 'EDUCATION_2020_07', 'EDUCATION_2020_08', 'EDUCATION_2020_09', 'EDUCATION_2020_10', 'EDUCATION_2020_11', 'EDUCATION_2020_12', 'EDUCATION_2021_01', 'EDUCATION_2021_02', 'EDUCATION_2021_03', 'EDUCATION_2021_04', 'EDUCATION_2021_05', 'EDUCATION_2021_06', 'EDUCATION_2021_07', 'EDUCATION_2021_08', 'EDUCATION_2021_09', 'EDUCATION_2021_10', '

In [27]:

# 对合并后的数据进行归一化
def normalize_social_factors(df):
    # 找出所有需要归一化的列（不包括 'COD_DIS', 'COD_BAR', 'Date', 'accident_rate_melted'）
    factor_columns = [col for col in df.columns if col not in ['COD_DIS', 'COD_BAR', 'Date', 'accident_rate_melted','PANDEMIC']]
    
    # 对每一列进行归一化
    for col in factor_columns:
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = (df[col] - min_val) / (max_val - min_val)
        
    return df

# 执行归一化操作并直接替换原数据
traffic_df = normalize_social_factors(traffic_df)

# 打印归一化后的数据
print(traffic_df.head())

   COD_DIS  COD_BAR     Date  accident_rate_melted  EDUCATION    INCOME  \
0        1       11  2018-01                 13.05   0.734139  0.372007   
1        1       12  2018-01                  6.82   0.605152  0.372007   
2        1       13  2018-01                 25.65   0.801392  0.372007   
3        1       14  2018-01                 25.57   0.820755  0.372007   
4        1       15  2018-01                 16.03   0.748281  0.372007   

   POPULATION  TRANSPORT  COMMERCIAL  RESIDENTIAL  PANDEMIC  
0    0.337342   0.464719    0.249695     0.317589         0  
1    0.948914   0.448354    0.284157     0.808627         0  
2    0.383697   0.784132    0.413408     0.648305         0  
3    0.502189   0.461904    0.492380     0.789608         0  
4    0.724325   0.605172    0.713202     0.817874         0  


In [28]:
# 确保 did_df 中的 Date 列为 'YYYY-MM' 格式
did_df['Date'] = did_df['year'].astype(str) + '-' + did_df['month'].astype(str).str.zfill(2)

# 检查 traffic_df 和 did_df 的 Date 列，确保格式一致
print("Unique dates in traffic_df:", traffic_df['Date'].unique())  # 查看 traffic_df 中的日期格式
print("Unique dates in did_df:", did_df['Date'].unique())  # 查看 did_df 中的日期格式

Unique dates in traffic_df: ['2018-01' '2018-02' '2018-03' '2018-04' '2018-05' '2018-06' '2018-07'
 '2018-08' '2018-09' '2018-10' '2018-11' '2018-12' '2019-01' '2019-02'
 '2019-03' '2019-04' '2019-05' '2019-06' '2019-07' '2019-08' '2019-09'
 '2019-10' '2019-11' '2019-12' '2020-01' '2020-02' '2020-03' '2020-04'
 '2020-05' '2020-06' '2020-07' '2020-08' '2020-09' '2020-10' '2020-11'
 '2020-12' '2021-01' '2021-02' '2021-03' '2021-04' '2021-05' '2021-06'
 '2021-07' '2021-08' '2021-09' '2021-10' '2021-11' '2021-12' '2022-01'
 '2022-02' '2022-03' '2022-04' '2022-05' '2022-06' '2022-07' '2022-08'
 '2022-09' '2022-10' '2022-11' '2022-12']
Unique dates in did_df: ['2018-01' '2018-02' '2018-03' '2018-04' '2018-05' '2018-06' '2018-07'
 '2018-08' '2018-09' '2018-10' '2018-11' '2018-12' '2019-01' '2019-02'
 '2019-03' '2019-04' '2019-05' '2019-06' '2019-07' '2019-08' '2019-09'
 '2019-10' '2019-11' '2019-12' '2020-01' '2020-02' '2020-03' '2020-04'
 '2020-05' '2020-06' '2020-07' '2020-08' '2020-09' '20

In [29]:
# 处理政策处理信息（合并到主数据）
did_df['Date'] = did_df['year'].astype(str) + '-' + did_df['month'].astype(str).str.zfill(2)

# 检查合并数据的列名和缺失值
print("Missing values in merged data before did merge:\n", traffic_df.isnull().sum())

traffic_df = pd.merge(traffic_df, did_df[['COD_DIS', 'COD_BAR', 'Date', 'treati', 'postit', 'DID']], 
                      on=['COD_DIS', 'COD_BAR', 'Date'], how='left')

# 检查合并后的数据
print("Missing values after did merge:\n", traffic_df.isnull().sum())

Missing values in merged data before did merge:
 COD_DIS                 0
COD_BAR                 0
Date                    0
accident_rate_melted    0
EDUCATION               0
INCOME                  0
POPULATION              0
TRANSPORT               0
COMMERCIAL              0
RESIDENTIAL             0
PANDEMIC                0
dtype: int64
Missing values after did merge:
 COD_DIS                 0
COD_BAR                 0
Date                    0
accident_rate_melted    0
EDUCATION               0
INCOME                  0
POPULATION              0
TRANSPORT               0
COMMERCIAL              0
RESIDENTIAL             0
PANDEMIC                0
treati                  0
postit                  0
DID                     0
dtype: int64


In [30]:
# 确保没有空值，去除包含空值的行（根据需要调整）
traffic_df = traffic_df.dropna()

# 设置DiD模型公式
formula = 'accident_rate_melted ~ DID + EDUCATION + INCOME + POPULATION + TRANSPORT + COMMERCIAL + RESIDENTIAL + PANDEMIC + C(COD_BAR) + C(Date)'

# 使用`statsmodels`进行回归分析
model = smf.ols(formula=formula, data=traffic_df).fit()

# 保存结果到Excel文件
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    model_summary = model.summary2().tables[1]
    model_summary.to_excel(writer, sheet_name='DiD_Model_Results')
     
print(f'输出已保存至: {output_file}')

输出已保存至: output_directory\DiD_model_results_2025-03-27_16-00-38.xlsx


In [31]:
    # 计算模型统计指标
model_stats = {
        "R-squared": model.rsquared,
        "Adj. R-squared": model.rsquared_adj,
        "F-statistic": model.fvalue,
        "Prob (F-statistic)": model.f_pvalue,
        "AIC": model.aic,
        "BIC": model.bic
    }
    # 保存结果到 Excel

stats_df = pd.DataFrame(model_stats.items(), columns=['Statistic', 'Value'])

print (stats_df.head())

            Statistic         Value
0           R-squared      0.688047
1      Adj. R-squared      0.680110
2         F-statistic     86.686091
3  Prob (F-statistic)      0.000000
4                 AIC  57641.865478
